In [1]:
# Import necessary libraries
import os, pickle, sys
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import pandas as pd
import seaborn as sns
from tqdm import tqdm

## Load Dataset

In [2]:
# Load dataset from pickle file
file = open('./emg_data.pkl', 'rb')
dataset = pickle.load(file)
file.close()

In [3]:
len(dataset)

18900

In [4]:
dataset[0]

('sub03',
 'D3',
 'S1',
 'OH',
 1,
           ch1       ch2       ch3       ch4       ch5       ch6       ch7  \
 0    0.414062  0.601562 -0.203125 -0.210938 -0.101562 -0.046875  0.257812   
 1    0.320312 -0.140625  0.070312  0.062500 -0.015625  0.062500 -0.296875   
 2   -0.476562 -0.140625 -0.593750  0.132812  0.039062  0.109375  0.015625   
 3   -0.039062 -0.492188 -0.679688 -0.031250  0.031250  0.062500 -0.101562   
 4   -0.007812 -0.093750  0.562500 -0.023438 -0.070312 -0.437500 -0.382812   
 ..        ...       ...       ...       ...       ...       ...       ...   
 795  0.054688 -0.101562  0.023438  0.000000  0.000000  0.000000 -0.015625   
 796 -0.093750 -0.046875 -0.054688 -0.023438 -0.007812 -0.023438  0.187500   
 797  0.046875 -0.054688  0.046875  0.039062  0.031250 -0.015625  0.250000   
 798  0.078125 -0.039062  0.046875  0.000000  0.000000 -0.023438 -0.031250   
 799  0.031250  0.117188  0.062500 -0.007812 -0.023438 -0.070312 -0.226562   
 
           ch8  
 0   -0.21

In [7]:
('OH' != 'IN') or ('OH' != 'GR')

True

In [11]:
[x for x in range(10) if x != 4]

[0, 1, 2, 3, 5, 6, 7, 8, 9]

In [14]:
dataset[0][3]

'OH'

In [23]:
dataset = [x for x in dataset if (x[3] != 'IN') and (x[3] != 'GR')]

In [24]:
len(dataset)

14700

## Create Windows

In [ ]:
fs = 200
data_n = 800
ts = np.linspace(0, data_n * (1/fs), data_n)
window_l = int(.20 * fs)
step = int(.029 *fs)

In [15]:
fs = 200
data_n = 800
ts = np.linspace(0, data_n * (1/fs), data_n)
window_l = int(.25 * fs)
step = int(.0625 *fs)

In [25]:
fs = 200
data_n = 800
ts = np.linspace(0, data_n * (1/fs), data_n)
window_l = int(.150 * fs)
step = int(.025 *fs)

In [26]:
window_n = (data_n - window_l) // step + 1

In [27]:
window_n

155

In [28]:
window_data = list()

# Process each EMG data in the dataset
for data in tqdm(dataset):
    subject = data[0]
    day = data[1]
    session = data[2]
    motion = data[3]
    repetition = data[4]
    emg = data[5]

    # Break down each EMG data into window-sized data
    for i in range(window_n):
        window = emg[i*step:(i*step)+window_l].to_numpy()
        window_data.append((subject, day, session, motion, repetition, window,))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14700/14700 [00:17<00:00, 860.44it/s]


In [29]:
df = pd.DataFrame(window_data, columns=['subject', 'day', 'session', 'motion', 'repetition', 'window_emg'])

In [30]:
len(df)

2278500

In [21]:
len(df)

1190700

In [31]:
df.head()

,subject,day,session,motion,repetition,window_emg
0,sub03,D3,S1,OH,1,"[[0.4140625, 0.6015625, -0.203125, -0.2109375,..."
1,sub03,D3,S1,OH,1,"[[-0.453125, 0.109375, -0.234375, -0.078125, -..."
2,sub03,D3,S1,OH,1,"[[-0.046875, 0.0390625, -0.4296875, -0.0546875..."
3,sub03,D3,S1,OH,1,"[[0.15625, 0.015625, -0.2265625, -0.0234375, 0..."
4,sub03,D3,S1,OH,1,"[[-0.1484375, -0.140625, 0.015625, 0.03125, 0...."


In [32]:
df.repetition.value_counts()

repetition
1     227850
2     227850
3     227850
4     227850
5     227850
6     227850
7     227850
8     227850
9     227850
10    227850
Name: count, dtype: int64

In [39]:
df.loc[0].window_emg.shape

(30, 8)

In [2]:
file = open('data-150-025.pkl', 'rb')
df = pickle.load(file)
file.close()

In [33]:
file = open('data-150-025.pkl', 'wb')
pickle.dump(df, file)
file.close()

In [24]:
file = open('data-250-0625.pkl', 'wb')
pickle.dump(df, file)
file.close()

In [14]:
file = open('data-250-25.pkl', 'wb')
pickle.dump(df, file)
file.close()

In [12]:
file = open('data-250-125.pkl', 'wb')
pickle.dump(df, file)
file.close()

In [2]:
file = open('data-250-125.pkl', 'rb')
df = pickle.load(file)
file.close()

In [11]:
file = open('data-200-29-2.pkl', 'wb')
pickle.dump(df, file)
file.close()

In [2]:
file = open('data-200-29.pkl', 'rb')
df = pickle.load(file)
file.close()

In [5]:
df.loc[0].window_emg.shape

(40, 8)

## Build Features

In [3]:
tqdm.pandas(desc='Building feature')

### Mean Absolute Value (MAV)

In [4]:
def mav(data):
    """
    Calculate the mean of the absolute values
    of the whole signal data
    """
    return np.mean(np.abs(data), axis=0)

In [5]:
mav(df.loc[0]['window_emg']).shape

(8,)

In [6]:
df['mav'] = df.window_emg.progress_apply(mav)

Building feature: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2278500/2278500 [00:08<00:00, 253688.09it/s]


### Waveform Length (WL)

In [7]:
def wl(data):
    """
    Calculate the sum absolutes differences
    of each values in the whole signal data
    """
    return np.sum(np.abs(np.diff(data, axis=0)), axis=0)

In [8]:
wl(df.loc[0]['window_emg']).shape

(8,)

In [9]:
df['wl'] = df.window_emg.progress_apply(wl)

Building feature: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2278500/2278500 [00:10<00:00, 217759.50it/s]


### Slope Sign Change (SSC)

In [10]:
def ssc(data, thresh=1e-6):
    """
    Calculate the number of times the slope
    of the waveform changes sign in the 
    entire signal data, with threshold value
    for the absolute difference between two 
    data points
    """
    diff = np.diff(data, axis=0)
    slope_mult = diff[:-1] * diff[1:] # negative if slope changes
    thresh_diff = (np.abs(diff[:-1]) >= thresh) | (np.abs(diff[1:]) >= thresh)

    return ((slope_mult < 0) & thresh_diff).sum(axis=0) # sum all slope changes above threshold

In [11]:
ssc(df.loc[0]['window_emg']).shape

(8,)

In [12]:
df['ssc'] = df.window_emg.progress_apply(ssc)

Building feature: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2278500/2278500 [00:17<00:00, 132647.68it/s]


### Zero Crossing (ZC)

In [13]:
def zc(data, thresh=1e-6):
    """
    Calculate the number of times the waveform
    crosses zero, with threshold value for the
    absolute difference between two data points
    """
    val_mult = data[:-1] * data[1:] # negative if crossing zero
    is_above_thresh = np.abs(np.diff(data, axis=0)) >= thresh

    return ((val_mult < 0) & is_above_thresh).sum(axis=0)

In [14]:
zc(df.loc[0]['window_emg']).shape

(8,)

In [15]:
df['zc'] = df.window_emg.progress_apply(zc)

Building feature: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2278500/2278500 [00:14<00:00, 158231.46it/s]


In [16]:
df.head()

,subject,day,session,motion,repetition,window_emg,mav,wl,ssc,zc
0,sub03,D3,S1,OH,1,"[[0.4140625, 0.6015625, -0.203125, -0.2109375,...","[0.24947916666666667, 0.17916666666666667, 0.1...","[10.1953125, 7.6640625, 9.4296875, 1.625, 1.35...","[16, 14, 24, 15, 15, 19, 19, 25]","[12, 15, 17, 16, 10, 15, 20, 18]"
1,sub03,D3,S1,OH,1,"[[-0.453125, 0.109375, -0.234375, -0.078125, -...","[0.24453125, 0.15338541666666666, 0.15390625, ...","[9.5859375, 6.8828125, 7.5234375, 1.25, 1.4296...","[16, 16, 23, 15, 17, 20, 20, 25]","[13, 14, 16, 17, 11, 15, 20, 18]"
2,sub03,D3,S1,OH,1,"[[-0.046875, 0.0390625, -0.4296875, -0.0546875...","[0.24869791666666666, 0.12395833333333334, 0.1...","[10.34375, 6.0859375, 6.7734375, 1.1875, 1.437...","[17, 18, 23, 16, 16, 17, 18, 25]","[13, 12, 16, 17, 12, 15, 19, 14]"
3,sub03,D3,S1,OH,1,"[[0.15625, 0.015625, -0.2265625, -0.0234375, 0...","[0.25052083333333336, 0.11614583333333334, 0.1...","[10.359375, 5.5859375, 7.0546875, 1.1484375, 1...","[18, 18, 24, 18, 14, 16, 18, 25]","[15, 11, 19, 17, 13, 13, 19, 15]"
4,sub03,D3,S1,OH,1,"[[-0.1484375, -0.140625, 0.015625, 0.03125, 0....","[0.2716145833333333, 0.10260416666666666, 0.11...","[11.390625, 4.390625, 6.171875, 1.0078125, 1.2...","[18, 16, 24, 17, 15, 14, 20, 25]","[15, 9, 17, 16, 12, 14, 21, 17]"


In [17]:
file = open('df_hudgin_150_025.pkl', 'wb')
pickle.dump(df, file)
file.close()

In [17]:
file = open('df_hudgin_250_125.pkl', 'wb')
pickle.dump(df, file)
file.close()

### Sample Entropy (SampEn)

In [280]:
def sampen(data, m, r): # TODO: confirm this function
    """
    SampEn = -log(A/B)
    More info on the calculation: 
    https://ieeexplore.ieee.org/document/4058892
    """
    N, channel_n = data.shape
    res = []

    for i in range(channel_n):
        data_ch = data.T[i]
        # Split time series and save all templates of length m and m+1
        xm = np.array([data_ch[i:i+m] for i in range(N-m)])
        xmp = np.array([data_ch[i:i+m+1] for i in range(N-(m+1))])
    
        # Calculate probability of vector matches
        B = np.sum([(np.sum(np.abs(xm - xmi).max(axis=1) <= (r * np.std(xmi))) - 1) / (N - m - 1) for xmi in xm]) / (N - m)
        A = np.sum([(np.sum(np.abs(xmp - xmi).max(axis=1) <= (r * np.std(xmi))) - 1) / (N - m - 2) for xmi in xmp]) / (N - m - 1)

        # Calculate sample entropy
        log = (-np.log(A/B)).tolist() if (A != 0.) and (B != 0) else 0.
        res.append(log)

    return np.array(res)

In [279]:
sampen(df.loc[0]['window_emg'], 2, .2)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 290.02it/s]


array([0.        , 0.        , 1.5668783 , 0.        , 0.        ,
       0.9870598 , 0.65058757, 1.05605267])

In [283]:
tqdm.pandas(desc='Sample Entropy Bar')

In [285]:
df['sampen'] = df.window_emg.progress_apply(sampen, m=2, r=.2)

Sample Entropy Bar: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 602910/602910 [1:42:41<00:00, 97.85it/s]


In [286]:
file = open('df_features.pkl', 'wb')
pickle.dump(df, file)
file.close()

In [288]:
df.head()

,subject,day,session,motion,window_emg,mav,wl,ssc,zc,mfl,rms,sampen
0,sub03,D3,S1,OH,"[[0.4140625, 0.6015625, -0.203125, -0.2109375,...","[0.26875, 0.1434375, 0.17390625, 0.03375, 0.03...","[18.875, 10.3515625, 14.3125, 2.296875, 2.1562...","[29, 27, 40, 29, 26, 29, 34, 42]","[24, 20, 30, 27, 18, 25, 35, 30]","[0.5745847396290436, 0.3028283336099577, 0.418...","[0.33822621521143215, 0.19764235376052372, 0.2...","[0.0, 0.0, 1.5668782980153044, 0.0, 0.0, 0.987..."
1,sub03,D3,S1,OH,"[[0.2421875, -0.0546875, -0.0078125, 0.015625,...","[0.2403125, 0.09796875, 0.135, 0.0196875, 0.02...","[18.3125, 6.9453125, 12.1015625, 1.4375, 2.218...","[33, 31, 41, 29, 33, 28, 37, 34]","[28, 19, 35, 21, 26, 25, 36, 28]","[0.5382523100435087, 0.1411510968454964, 0.304...","[0.29961726888231927, 0.12974866780385455, 0.1...","[0.0, 0.0, 1.4043593685175295, 1.5668782980153..."
2,sub03,D3,S1,OH,"[[-0.5546875, -0.125, -0.0625, -0.0234375, -0....","[0.153125, 0.1021875, 0.130625, 0.01890625, 0....","[11.6640625, 7.6328125, 12.078125, 1.3125, 1.9...","[31, 31, 40, 29, 29, 25, 37, 25]","[29, 26, 37, 18, 25, 22, 29, 24]","[0.32113333361471874, 0.13323166812608592, 0.3...","[0.1945422311157657, 0.12800108336943872, 0.17...","[2.5223897430427407, 0.0, 0.0, 0.0, 2.26002547..."
3,sub03,D3,S1,OH,"[[-0.1953125, -0.0625, -0.265625, -0.0234375, ...","[0.12859375, 0.10546875, 0.10609375, 0.0170312...","[9.609375, 8.34375, 8.875, 1.140625, 1.375, 5....","[30, 32, 37, 34, 22, 28, 34, 30]","[30, 27, 32, 17, 16, 24, 26, 28]","[0.237026638143589, 0.1690389659258844, 0.2200...","[0.16408853960019573, 0.1365601758955553, 0.14...","[1.9033505346365174, 0.0, 2.1546649629174235, ..."
4,sub03,D3,S1,OH,"[[0.0703125, -0.1484375, 0.171875, -0.015625, ...","[0.13140625, 0.1171875, 0.08609375, 0.01421875...","[9.59375, 9.6875, 6.8046875, 0.9296875, 1.1796...","[33, 32, 35, 33, 25, 32, 33, 35]","[26, 32, 28, 15, 16, 26, 30, 29]","[0.25211559711606946, 0.2316208764883889, 0.08...","[0.16966260752041093, 0.14889731413376806, 0.1...","[0.6505875661411494, 0.0, 0.9129518306086403, ..."


### Approximate Entropy (ApEn)

### Maximum Fractal Length (MFL)

In [217]:
np.isneginf([1, 2, np.inf, -np.inf])

array([False, False, False,  True])

In [220]:
def mfl(data):
    """
    Calculate the log10 of the square-root
    of the sum of the squared difference
    of the entire signal data values
    """
    res = np.log10(np.sqrt(np.sum(np.diff(data, axis=0)**2, axis=0)))
    res[np.isneginf(res)] = 0. # TODO: confirm this imputation
    return res

In [125]:
mfl(df.loc[0]['window_emg']).shape

(8,)

In [221]:
df['mfl'] = df.window_emg.apply(mfl)

/var/folders/fg/wwsz02rn0qn31sz3qslnnjm80000gn/T/ipykernel_5496/2237827611.py:7: RuntimeWarning: divide by zero encountered in log10
  res = np.log10(np.sqrt(np.sum(np.diff(data, axis=0)**2, axis=0)))


In [223]:
df.mfl[0].shape

(8,)

### Root Mean Square (RMS)

In [122]:
def rms(data):
    """
    Calculate the root mean square of the
    entire signal data
    """
    return np.sqrt(np.mean(data**2, axis=0))

In [123]:
rms(df.loc[0]['window_emg']).shape

(8,)

In [222]:
df['rms'] = df.window_emg.apply(rms)

In [224]:
df.rms[0].shape

(8,)

### Cepstral Coefficient (CC)

In [293]:
def cc(data, order=4):
    """
    
    """
    def _cc_window(window):
        spectrum = np.fft.fft(window)
        cepstrum = np.fft.ifft(np.log(np.abs(spectrum)))
        return np.real(cepstrum[:order])
    
    return np.apply_along_axis(_cc_window, axis=0, arr=data)

In [295]:
a = np.fft.fft(df.loc[0]['window_emg'][0])
b = np.fft.ifft(np.log(np.abs(a)))
np.real(b[:4])

array([-0.29256724,  0.10458447, -0.27049019,  0.08483695])

In [296]:
cc(df.loc[0]['window_emg']).shape

(4, 8)

In [304]:
df['cc'] = df.window_emg.progress_apply(cc)

Sample Entropy Bar:   0%|                                                                                                                                                                                                                                                | 0/602910 [00:00<?, ?it/s]/var/folders/fg/wwsz02rn0qn31sz3qslnnjm80000gn/T/ipykernel_5496/2346512558.py:7: RuntimeWarning: divide by zero encountered in log
  cepstrum = np.fft.ifft(np.log(np.abs(spectrum)))
Sample Entropy Bar:   1%|██▋                                                                                                                                                                                                                                | 7269/602910 [00:00<01:02, 9537.71it/s]/opt/homebrew/Caskroom/miniforge/base/envs/emg-kcl/lib/python3.12/site-packages/numpy/fft/_pocketfft.py:94: RuntimeWarning: invalid value encountered in ifft
  return ufunc(a, fct, axes=[(axis,), (), (axis,)], out=out)
Sample Entropy 

In [303]:
cc(df.loc[2]['window_emg'])

array([[ 1.20682915e-01, -4.31064026e-01, -2.68057755e-01,
        -2.02758318e+00, -1.75510934e+00, -7.75864354e-01,
         4.15363006e-01, -2.82363852e-01],
       [-1.43668177e-01, -2.75476765e-02, -3.27451515e-01,
        -4.09764261e-02, -8.37505065e-02, -3.23295534e-02,
        -3.15795716e-01, -3.10050201e-01],
       [-7.43320022e-02,  1.34198712e-01, -2.94529651e-02,
        -1.43173170e-01,  1.83097795e-03, -1.46372735e-01,
         2.33716447e-01, -8.07544299e-02],
       [ 2.35666463e-02,  3.71957430e-02, -1.05487050e-01,
        -3.37275580e-02, -2.05216499e-01, -1.48701669e-01,
         1.62382073e-01, -1.07381113e-01]])

In [309]:
file = open('df_features.pkl', 'wb')
pickle.dump(df, file)
file.close()